In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go

In [2]:
# Pegar a ação 
acao = yf.Ticker('PETR4.SA')
data = acao.history(period='1y')
df = data[['Close']]

In [3]:
df

,Close
Date,
2021-07-19,18.083031
2021-07-20,18.324230
2021-07-21,18.579210
2021-07-22,18.537863
2021-07-23,18.427599
...,...
2022-07-13,28.250000
2022-07-14,27.490000
2022-07-15,27.959999


In [4]:
# Calcular media movel
mm = df.rolling(window=20).mean()
mm

,Close
Date,
2021-07-19,NaN
2021-07-20,NaN
2021-07-21,NaN
2021-07-22,NaN
2021-07-23,NaN
...,...
2022-07-13,27.9480
2022-07-14,27.8685
2022-07-15,27.9010


In [5]:
# SD
dp = df.rolling(window=20).std()
dp

,Close
Date,
2021-07-19,NaN
2021-07-20,NaN
2021-07-21,NaN
2021-07-22,NaN
2021-07-23,NaN
...,...
2022-07-13,0.796516
2022-07-14,0.755898
2022-07-15,0.744509


In [6]:
# Calculo banda sup e inf
sup_band = mm + 2 * dp
inf_band = mm - 2 * dp

In [7]:
# Alterar nome das colunas para sup e inf
sup_band = sup_band.rename(columns = {'Close': 'Superior'})
inf_band = inf_band.rename(columns = {'Close': 'Inferior'})

In [8]:
# Unir colunas
bandas_bollinger = df.join(sup_band).join(inf_band)

In [9]:
bandas_bollinger

,Close,Superior,Inferior
Date,,,
2021-07-19,18.083031,NaN,NaN
2021-07-20,18.324230,NaN,NaN
2021-07-21,18.579210,NaN,NaN
2021-07-22,18.537863,NaN,NaN
2021-07-23,18.427599,NaN,NaN
...,...,...,...
2022-07-13,28.250000,29.541032,26.354968
2022-07-14,27.490000,29.380296,26.356704
2022-07-15,27.959999,29.390018,26.411982


In [10]:
# Retirar nulos
bandas_bollinger.dropna(inplace=True)

In [11]:
# Calculo pontos de compra e venda
compra = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['Inferior']]
venda = bandas_bollinger[bandas_bollinger['Close'] <= bandas_bollinger['Superior']]

In [12]:
pio.templates.default = 'plotly_dark'

fig = go.Figure()
fig.add_trace(go.Scatter(x=inf_band.index,
                        y=inf_band['Inferior'],
                        name='Banda_Inferior',
                        line_color='rgba(173,204,255,0.2)'
                        ))

fig.add_trace(go.Scatter(x=sup_band.index,
                        y=sup_band['Superior'],
                        name='Banda_Superior',
                        fill='tonexty',
                        fillcolor='rgba(173,204,255,0.2)',
                        line_color='rgba(173,204,255,0.2)'
                        ))

fig.add_trace(go.Scatter(x=df.index,
                        y=df['Close'],
                        name='Preco Fechamento',
                        line_color='#636EFA'
                        ))

fig.add_trace(go.Scatter(x=mm.index,
                        y=mm['Close'],
                        name='Media Movel',
                        line_color='#FECB52'
                        ))

fig.add_trace(go.Scatter(x=compra.index,
                        y=compra['Close'],
                        name='Compra',
                        mode='markers',
                        marker=dict(
                            color='#00CC96',
                            size=8,
                            )
                        
                        ))
fig.add_trace(go.Scatter(x=venda.index,
                        y=venda['Close'],
                        name='Venda',
                        mode='markers',
                        marker=dict(
                            color='#EF553B',
                            size=8,
                            )
                        
                        ))
fig.show()